# リンク予測(DrugBank)
## ECFP -> ニューラルネットワーク

In [2]:
import pandas as pd
import numpy as np

from rdkit import Chem
from rdkit.Chem import AllChem, Draw

In [3]:
import torch
import torch.utils.data
from torchvision import datasets
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim  # 最適化アルゴリズム実装のためのライブラリ

In [4]:
df_link = pd.read_pickle('link_pare.pkl')

In [5]:
df_nolink = pd.read_pickle('nolink_pare.pkl')

In [6]:
df_link.tail()

,smiles_1,smiles_2,target
650632,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
650633,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
650634,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
650635,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
650636,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1


In [7]:
df_nolink.tail()

,smiles_1,smiles_2,target
2582384,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0
2582385,"[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0
2582386,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0
2582387,"[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2582388,"[0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0


In [8]:
class Net(nn.Module):  # 多層ニューラルネットワークの構築
    def __init__(self):
        super(Net, self).__init__()
        self.share1 = nn.Linear(512,1024)
        self.share2 = nn.Linear(1024,512)
        self.fc1 = nn.Linear(1024, 128)  # 一つ目の隠れ層のユニット数は512
        self.fc2 = nn.Linear(128, 128)  # 二つ目の隠れ層のユニット数は128
        self.fc3 = nn.Linear(128, 1)  # 出力層のユニット数は1


    def forward(self, x, y):
        a = F.relu(self.share1(x))
        a = self.share2(a)
        b = F.relu(self.share1(y))
        b = self.share2(b)
        x = torch.cat([a,b], dim=1)
        x = F.relu(self.fc1(x))  # 活性化関数にはReLUを使用
        x = F.relu(self.fc2(x))  # 活性化関数にはReLUを使用
        x = torch.sigmoid(self.fc3(x))
        return x,a,b

net = Net()

In [9]:
net.train()

Net(
  (share1): Linear(in_features=512, out_features=1024, bias=True)
  (share2): Linear(in_features=1024, out_features=512, bias=True)
  (fc1): Linear(in_features=1024, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=1, bias=True)
)

In [10]:
df = pd.concat([df_link, df_nolink])
df = df.sample(frac=1)
df = df.reset_index(drop=True)

In [11]:
df_sample = df.sample(n=10000)
df_sample = df_sample.reset_index(drop=True)

In [12]:
df_sample.head()

,smiles_1,smiles_2,target
0,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
2,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",1
3,"[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0
4,"[0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...",1


In [13]:
A = df_sample["smiles_1"]
B = df_sample["smiles_2"]
target = df_sample["target"]

In [14]:
A = torch.Tensor(A)
B = torch.Tensor(B)
target = torch.Tensor(target)

In [15]:
train_tensor = torch.utils.data.TensorDataset(A,B,target)
train_dataset, test_dataset = torch.utils.data.random_split(train_tensor, [9000, 1000])

In [16]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [17]:
criterion = nn.BCELoss() 
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)
#optimizer = optim.Adam(net.parameters(), lr=1e-3, weight_decay=1e-5)

In [20]:
def HSIC(a, b, sigma):
    K = gauss_kernel(a,sigma)
    L = gauss_kernel(b,sigma)

    KH = K - K.mean(0, keepdim=True)
    LH = L - L.mean(0, keepdim=True)

    N = len(a)

    return torch.trace(KH @ LH / (N - 1) ** 2)

In [19]:
def gauss_kernel(X, sigma):
        X = X.view(len(X), -1)
        XX = X @ X.t()
        X_sqnorms = torch.diag(XX)
        X_L2 = -2 * XX + X_sqnorms.unsqueeze(1) + X_sqnorms.unsqueeze(0)
        gamma = 1 / (2 * sigma ** 2)

        kernel_XX = torch.exp(-gamma * X_L2)
        return kernel_XX    

In [ ]:
sigma = 0.05

In [22]:
for epoch in range(100):
    total_loss = 0.0
    for i,data in enumerate(train_loader):
        inputA,inputB, labels = data
        optimizer.zero_grad()  
        outputs,a,b = net(inputA,inputB)
        loss = criterion(outputs, labels) + HSIC(a,b,sigma)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(total_loss)
print('\n学習が終了しました。')

148.68605606257915
148.4205640554428
147.98915591835976
147.57437726855278
146.97290961444378
146.13596218824387
145.519066542387
144.4320888221264
142.63410639762878
140.44207108020782
138.57589656114578
136.174987077713
133.17317774891853
130.19369640946388
126.49231521785259
122.88634391129017
118.95725256204605
115.00519524514675
110.77730922400951
106.53470186889172
101.85400372743607
97.33821001648903
92.52382262051105
88.58387511968613
85.01986136287451
81.9556042701006
78.69864957034588
74.83836305141449
72.65482620149851
70.26820005849004
70.07954581081867
66.11408842355013
65.91660421341658
62.10278160125017
61.90844193845987
60.64695828408003
58.161074347794056
57.86648181825876
56.87738758325577
56.42499742656946
54.16445507109165
52.657872438430786
55.70861084386706
52.10247366502881
54.053269885480404
50.75604601204395
49.09364575892687
50.502960063517094
49.11621892079711
47.31372928619385
46.8487930893898
47.732965379953384
47.61028918996453
46.104825634509325
46.108056

In [38]:
correct = 0.0  # 正答数を表す
total = 0.0  # テストデータの総数を表す
link = 0.0

In [39]:
y_label = np.empty(0)
y_predict = np.empty(0)

In [40]:
for data in test_loader:
    inputA, inputB, labels = data
    outputs,a,b = net(inputA,inputB)
    _, predicted = torch.max(outputs.data, 0)
    total += labels.size(0)  # テストデータの総数を計算
    correct += (predicted == labels).sum().item()  # 正答数を計算
    link += sum(labels.numpy())
    y_label = np.concatenate([y_label, np.array(labels.numpy())])
    y_predict = np.concatenate([y_predict, np.array(outputs.detach().numpy()).ravel()])

In [41]:
pred = []
for i in y_predict:
    if i >= 0.5:
        pred.append(1)
    else:
        pred.append(0)

In [42]:
count = 0
for i in range(1000):
    if y_label[i] == pred[i]:
        count += 1
count

814

In [43]:
print('テストデータに対する正答率： %d / %d = %f' % (count, total, count / total) + '\n')
print('テストデータに含まれるリンクありの割合： %d / %d = %f' % (link, total, link / total) + '\n')
print('テストデータに含まれるリンクなしの割合： %d / %d = %f' % (total - link, total, (total- link) / total) + '\n')

for i in range(10):  # テストデータの一部を10行に分けて可視化
    print("ラベル：" + "".join('%d ' % y_label[i*10 + j] for j in range(10)))  # ラベルの値を表示
    print("　予測：" + "".join('%d ' % pred[i*10 + j] for j in range(10)) + "\n")  # 予測結果を表示

テストデータに対する正答率： 814 / 1000 = 0.814000

テストデータに含まれるリンクありの割合： 198 / 1000 = 0.198000

テストデータに含まれるリンクなしの割合： 802 / 1000 = 0.802000

ラベル：0 0 0 0 0 0 0 0 0 1 
　予測：0 0 0 0 0 0 0 0 0 0 

ラベル：1 0 0 0 1 0 0 0 0 0 
　予測：1 0 0 0 1 0 0 0 0 0 

ラベル：0 0 0 0 0 0 0 0 0 0 
　予測：0 1 1 0 0 0 0 0 0 0 

ラベル：1 0 0 0 0 1 0 1 0 1 
　予測：1 0 0 0 0 0 0 1 0 1 

ラベル：0 0 0 0 0 1 1 0 0 0 
　予測：0 0 0 0 0 1 1 0 0 0 

ラベル：0 0 0 0 1 1 0 1 1 0 
　予測：0 0 0 0 1 0 0 0 1 0 

ラベル：0 0 1 1 0 0 0 0 0 0 
　予測：0 0 1 1 0 1 0 1 1 0 

ラベル：0 0 0 1 0 0 0 0 0 0 
　予測：0 0 0 1 0 1 0 0 0 0 

ラベル：0 0 0 0 0 1 0 0 0 0 
　予測：1 0 0 0 0 1 0 0 0 0 

ラベル：0 0 0 0 1 0 0 1 0 0 
　予測：0 0 0 0 1 0 0 1 0 0 



In [44]:
from sklearn.metrics import roc_auc_score

In [45]:
roc_auc_score(y_label, y_predict)

0.8376470440061462